# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [60]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys_vacation import geoapify_key

In [61]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [62]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    x='Lng',
    y='Lat',
    c='Humidity',
    size='Humidity',
    hover_cols=['City', 'Country'],
    cmap='viridis',
    title='City Humidity Map',
    alpha=.75,
    tiles='OSM'
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [63]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] > 10) &
    (city_data_df['Max Temp'] < 40) &
    (city_data_df['Cloudiness'] < 10) &
    (city_data_df['Wind Speed'] < 15) &
    (city_data_df['Humidity'] < 50)
]
# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
29,29,saint-louis,16.3333,-15.0000,38.80,24,4,1.78,SN,1666108247
73,73,nalut,30.3333,10.8500,25.41,27,0,5.11,LY,1666108275
90,90,saint george,37.1041,-113.5841,13.33,44,0,1.54,US,1666108286
96,96,zhangye,38.9342,100.4517,11.11,16,0,1.67,CN,1666108292
115,115,cedar city,37.6775,-113.0619,14.11,49,0,0.00,US,1666108304
141,141,brenham,30.1669,-96.3977,14.47,40,0,3.09,US,1666108319
174,174,avera,33.1940,-82.5271,12.53,26,0,4.90,US,1666108342
176,176,challapata,-18.9000,-66.7667,14.15,30,0,5.26,BO,1666108343
231,231,carrollton,32.9537,-96.8903,14.63,31,0,5.66,US,1666108361
239,239,rawson,-43.3002,-65.1023,23.74,25,2,13.52,AR,1666108102


### Step 3: Create a new DataFrame called `hotel_df`.

In [65]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
29,saint-louis,SN,16.3333,-15.0000,24,
73,nalut,LY,30.3333,10.8500,27,
90,saint george,US,37.1041,-113.5841,44,
96,zhangye,CN,38.9342,100.4517,16,
115,cedar city,US,37.6775,-113.0619,49,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [72]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
saint-louis - nearest hotel: No hotel found
nalut - nearest hotel: No hotel found
saint george - nearest hotel: The Advenire, an Autograph Collection Hotel
zhangye - nearest hotel: Zhangye Hotel 张掖饭店
cedar city - nearest hotel: Baymont Inn & Suites
brenham - nearest hotel: Ant Street Inn
avera - nearest hotel: No hotel found
challapata - nearest hotel: Residencial Marian
carrollton - nearest hotel: Days Inn North Dallas/Farmers Branch
rawson - nearest hotel: Hotel Deportivo
comodoro rivadavia - nearest hotel: No hotel found
kingman - nearest hotel: Home2 Suites by Hilton Kingman
aktau - nearest hotel: Гульдана
lakki marwat - nearest hotel: No hotel found
statesville - nearest hotel: The Kerr House Bed & Breakfast
tikrit - nearest hotel: فندق بلازا
xuchang - nearest hotel: first hotel
bilma - nearest hotel: No hotel found
pali - nearest hotel: Hotel Pawan International


,City,Country,Lat,Lng,Humidity,Hotel Name
29,saint-louis,SN,16.3333,-15.0000,24,No hotel found
73,nalut,LY,30.3333,10.8500,27,No hotel found
90,saint george,US,37.1041,-113.5841,44,"The Advenire, an Autograph Collection Hotel"
96,zhangye,CN,38.9342,100.4517,16,Zhangye Hotel 张掖饭店
115,cedar city,US,37.6775,-113.0619,49,Baymont Inn & Suites
141,brenham,US,30.1669,-96.3977,40,Ant Street Inn
174,avera,US,33.1940,-82.5271,26,No hotel found
176,challapata,BO,-18.9000,-66.7667,30,Residencial Marian
231,carrollton,US,32.9537,-96.8903,31,Days Inn North Dallas/Farmers Branch
239,rawson,AR,-43.3002,-65.1023,25,Hotel Deportivo


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [77]:
%%capture --no-display

# Configure the map plot
map_plot1 = hotel_df.hvplot.points(
   x='Lng',
    y='Lat',
    c='Humidity',
    size='Humidity',
    hover_cols=["City", "Country", "Humidity", "Hotel Name"],
    cmap='viridis',
    title='City Hotel Map',
    alpha=.75,
    tiles='OSM',
    frame_width =  1000,
    frame_height = 600,
)

# Display the map
map_plot1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)